# Section 2-2 - Regularized Neural Network

In this section, we'll introduce two additional layers to our model. The first is called the rectified linear unit (hereafter, ReLU), which helps introduce non-linearity into the network. The second is called the dropout layer, which acts to regularize the network and prevent overfitting.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(1337)

df = pd.read_csv('data/mnist.csv')

In [2]:
df_train = df.iloc[:33600, :]

X_train = df_train.iloc[:, 1:].values / 255.
y_train = df_train['label'].values
y_train_onehot = pd.get_dummies(df_train['label']).values

In [3]:
df_test = df.iloc[33600:, :]

X_test = df_test.iloc[:, 1:].values / 255.
y_test = df_test['label'].values

## Benchmark

In [4]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=0, verbose=3)
model = model.fit(X_train, df_train['label'].values)

y_prediction = model.predict(X_test)
print "\naccuracy", np.sum(y_prediction == df_test['label'].values) / float(len(y_test))

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10

accuracy 0.937857142857


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


## Regularized NN

While matrix operations are linear, there could be a non-linear relationship between the features and the label. Introducing a ReLU layer, defined as f(x) = max(0, x), can help the model capture this interaction. ReLU is widely used as its simplicity allows for much faster training without a high cost to accuracy.

The dropout layer can be thought of as a form of sampling, where output values are randomly set to zero by a pre-specified probability. This creates a more robust network as the process prevents interdependence, and as such the model is less likely to overfit on the training data. It is surprisingly effective, which has made it an active area of research.

Following Andrej Karpathy's advice of "don't be a hero", the example shown here is from the Keras repository.

In [5]:
# https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

start = time()

model = Sequential()
model.add(Dense(512, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.fit(X_train, y_train_onehot)

print '\ntime taken %s seconds' % str(time() - start)

Using TensorFlow backend.


Epoch 1/10
33600/33600 [==============================] - 6s - loss: 0.8148 - acc: 0.7814     
Epoch 2/10
33600/33600 [==============================] - 6s - loss: 0.3841 - acc: 0.8886     
Epoch 3/10
33600/33600 [==============================] - 6s - loss: 0.3139 - acc: 0.9085     
Epoch 4/10
33600/33600 [==============================] - 7s - loss: 0.2741 - acc: 0.9196     
Epoch 5/10
33600/33600 [==============================] - 7s - loss: 0.2444 - acc: 0.9286     
Epoch 6/10
33600/33600 [==============================] - 7s - loss: 0.2184 - acc: 0.9361     
Epoch 7/10
33600/33600 [==============================] - 6s - loss: 0.1995 - acc: 0.9413     
Epoch 8/10
33600/33600 [==============================] - 7s - loss: 0.1840 - acc: 0.9456     
Epoch 9/10
33600/33600 [==============================] - 6s - loss: 0.1715 - acc: 0.9492     
Epoch 10/10
33600/33600 [==============================] - 7s - loss: 0.1565 - acc: 0.9536     

time taken 69.9661490917 seconds


In [6]:
y_prediction = model.predict_classes(X_test)
print "\n\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test))

8288/8400 [============================>.] - ETA: 0s

accuracy 0.957976190476


Introducing ReLU and dropout layers have enabled the model to outperform the benchmark by 2%! In the next section, we introduce new layers that take advantage of the 2D structure of the image to further improve model performance.